# __Analisis de Datos Históricos de consumo y precio en España (2000 - 2024)__

# Procesamiento de los datos oficiales.

https://ine.es/jaxiT3/Datos.htm?t=28189#_tabs-tabla

https://www.mapa.gob.es/es/alimentacion/temas/consumo-tendencias/panel-de-consumo-alimentario/series-anuales/default.aspx

Vamos a estar analizando la evolución del __Consumo x Cápita__ y del __Precio__ de los siguientes 9 alimentos:
- *CARNE DE VACUNO*
- *CARNE DE POLLO*
- *CARNE DE CERDO*
- *ACEITE DE OLIVA*
- *TOMATES*
- *SALMON*
- *ARROZ*
- *CONSERVAS*
- *PAN*

Primero de todo necesitamos importar las librerias de python que necesitaremos:

In [9]:
import openpyxl
import pandas as pd
from pathlib import Path

### Primero procesamos los datos alimentarios

Ahora debido a que los datos se encuentran divididos en __25 excels__, con las distintas variables en las distintas páginas del excel, he decidido crear este `script` para generar archivos csv que luego juntaré para crear la base de datos sobre la que posteriormente realizaré mi analisis. (El script tarda aproximadamente un minuto porque debe cargar y procesar 25 archivos Excel, lo cual es una operación relativamente pesada. Leer archivos `.xlsx` con pandas es más lento que leer otros formatos como `.csv`).

In [10]:
path = Path('../Raw Data')
files_list = [filename for filename in path.glob('*.xlsx')] #Creo una lista con todos los archivos del directorio Excel Data

df_consumo = pd.DataFrame(columns=['PRODUCTO','AÑO','CONSUMOXCAPITA']) #Aquí creo los dos dataframes finales a los que les voy a ir añadiendo filas
df_precio = pd.DataFrame(columns=['PRODUCTO','AÑO','PRECIO']) 


food_products = ['CARNE VACUNO','CARNE POLLO','CARNE CERDO','ACEITE DE OLIVA','TOMATES','SALMON','ARROZ','PLAT.PREP.CONSERVA','PAN'] #Alimentos a analizar

for filename in files_list:
    filewb = openpyxl.load_workbook(filename,data_only=True)  
    for sheetname in filewb.sheetnames:                   #Iteramos sobre las sheets de cada excel file
        if sheetname in ['CONSUMOXCAPITA','CONSUMOXCÁPITA','CONSUMO X CÁPITA','PRECIO','PRECIOS']:
            filedf = pd.read_excel(filename,sheet_name=sheetname,header = 2)
            filedf = filedf.rename(columns = {'Unnamed: 0':'PRODUCTO'}) 

            if 'PRECIO' in sheetname:
                filedf = filedf.rename(columns = {'.TOTAL ESPAÑA':'PRECIO','T.ESPAÑA':'PRECIO'},errors='ignore')
                filedf = filedf.loc[(filedf['PRODUCTO'].isin(food_products))|(filedf['PRODUCTO'].str.contains(r'.*ACEITE.*OLIVA.*')),['PRODUCTO','PRECIO']]
            else:
                filedf = filedf.rename(columns = {'.TOTAL ESPAÑA':'CONSUMOXCAPITA','T.ESPAÑA':'CONSUMOXCAPITA'},errors='ignore')
                filedf = filedf.loc[(filedf['PRODUCTO'].isin(food_products))|(filedf['PRODUCTO'].str.contains(r'.*ACEITE.*OLIVA.*')),['PRODUCTO','CONSUMOXCAPITA']]
            
            año = filename.name[:4]     # Homogeinizo y filtro las columnas que quiero seleccionar

            filedf['AÑO'] = int(año)
            
            if 'PRECIO' in sheetname:   # Ahora toca concatenar todos estos dfs a los dos dataframes finales df_consumo y df_precio: 
                df_precio = pd.concat([df_precio,filedf])
            else:
                df_consumo = pd.concat([df_consumo,filedf])




Finalmente utilizamos el método `merge()` y guardamos nuestro Dataframe final.

In [11]:
df_clean = pd.merge(df_precio,df_consumo,on=['AÑO','PRODUCTO'],how='inner')

Observando los datos originales, nos damos cuenta que hay dos productos con el mismo nombre *"TOMATES"* pero distintos datos ya que __el primero se refiere a TOMATES como hortaliza fresca y el segundo a TOMATES como conserva__.

Una vez decididos a eliminar TOMATES en conserva de nuestro dataframe solo queda usar `drop.duplicates()` ya que este siempre aparece en segundo lugar.

Además debemos homogeneizar el nombre de la suma de las diferentes variedades de aceite de oliva ya que de 2000-2012, este está etiquetado como *"ACEITE DE OLIVA"*, de 2013-2023 *"TOTAL ACEITES DE OLIVA"* y en 2024 como *"TOTAL ACEITES OLIVA"*.

In [12]:
df_clean = df_clean.drop_duplicates(subset=['AÑO','PRODUCTO']).copy()
df_clean['PRODUCTO']=df_clean['PRODUCTO'].replace({'ACEITE DE OLIVA':'ACEITE DE OLIVA',
                                               'TOTAL ACEITES DE OLIVA':'ACEITE DE OLIVA',
                                               'TOTAL ACEITES OLIVA':'ACEITE DE OLIVA'})

Por último guardamos el archivo final.

In [13]:
df_clean.to_csv('../Filtered Data/Datos_consumo_precio_espana.csv',index=False)

### Ahora vamos a procesar el archivo de ganancias medias anuales por trabajador de todas las edades y de ambos sexos (2008-2023) que extraje del INE:

In [14]:
salario=pd.read_csv('../Raw Data/28189.csv',sep=';')
salario.head()

,Grupos de edad,Sexo,Periodo,Total
0,Todas las edades,Ambos sexos,2023,"28.049,94"
1,Todas las edades,Ambos sexos,2022,"26.948,87"
2,Todas las edades,Ambos sexos,2021,"25.896,82"
3,Todas las edades,Ambos sexos,2020,"25.165,51"
4,Todas las edades,Ambos sexos,2019,"24.395,98"


In [15]:
salario = salario.rename(columns={'Total':'Ganancia','Periodo':'AÑO'})
salario = salario[['AÑO','Ganancia']]

In [16]:
salario.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   AÑO       16 non-null     int64 
 1   Ganancia  16 non-null     object
dtypes: int64(1), object(1)
memory usage: 384.0+ bytes


Aquí nos damos cuenta que Ganancia está formateado como un string y eso nos generará problemas mas tarde.

In [17]:
def reformat(a):
    a = a.replace('.','')
    a = a.replace(',','.')
    return float(a)

salario['Ganancia'] = salario['Ganancia'].apply(reformat)

In [18]:
salario.head()

,AÑO,Ganancia
0,2023,28049.94
1,2022,26948.87
2,2021,25896.82
3,2020,25165.51
4,2019,24395.98


In [19]:
salario.to_csv('../Filtered Data/salarios.csv',index=False)

Ahora que tenemos los datos filtrados y ordenados, ya podemos volcarnos en el análisis que nos concierne.